# Gold Price Prediction Using Machine Learning and Data Visualization

### Precious Metals Prices
- **Gold Spot (Target Variable):** The physical gold price in the global market, serving as the main prediction target.  
- **Gold Futures (GC=F):** Futures contracts reflecting market expectations of future gold prices.  
- **Silver Futures (SI=F):** Highly correlated with gold, representing a substitute and co-movement within precious metals.   

### Financial Market Indicators
- **S&P 500 Index & NASDAQ:** Major U.S. stock indices, often inversely correlated with gold during “risk-on” periods when investors prefer equities over safe havens.  
- **Crude Oil Prices (CL=F):** Oil price fluctuations influence inflation expectations and production costs, indirectly affecting gold.  
- **VIX (Volatility Index):** Measures expected market volatility; higher VIX typically raises demand for gold as a safe-haven asset.  
- **SPDR Gold Shares ETF (GLD ETF):** Represents gold holdings by investors through ETFs, reflecting direct financial investment demand for gold.  

### Macroeconomic Indicators
- **Consumer Price Index (CPI):** A key inflation measure. Higher inflation generally boosts gold demand as a store of value.  
- **Federal Funds Rate:** The benchmark interest rate set by the Federal Reserve. Higher rates reduce gold’s appeal since it yields no interest.  
- **10-Year Treasury Yield & Real Interest Rate:** Indicators of real returns on government bonds; lower real yields typically strengthen gold prices.  
- **U.S. Dollar Index (UUP):** Measures USD strength. A stronger dollar usually weakens gold prices as gold is dollar-denominated.  
- **M2 Money Supply:** Represents overall liquidity in the economy; rising money supply may fuel inflation expectations and support gold.  
- **Unemployment Rate:** Reflects overall economic health and investor sentiment. High unemployment may increase gold’s safe-haven demand.  

### Geopolitical Risk Indicators
- **Geopolitical Risk Index (GPR, GPRA, GPRT):** Measures global geopolitical tensions and uncertainty. Higher risk levels tend to increase demand for gold as a safe-haven asset.  

---


In [2]:
import ace_tools_open as tools;

## Market data

In [ ]:
import yfinance as yf

tickers = {
    'GC=F': 'Gold_Futures',
    'SI=F': 'Silver_Futures',
    'CL=F': 'Crude_Oil',
    '^GSPC': 'SP_500',
    '^IXIC': 'NASDAQ',
    'GLD': 'Gold_ETF'
}

# Download 'Close' price data
raw_data = yf.download(list(tickers.keys()), start='2000-01-01', auto_adjust=False)

# Extract Close prices
close_data = raw_data['Close'].copy()

# Rename columns to descriptive names
close_data = close_data.rename(columns=tickers)

# Save to CSV
close_data.to_csv("../data/raw/market_data.csv")

# Preview
tools.display_dataframe_to_user(name="market_data", dataframe=close_data)


[*********************100%***********************]  6 of 6 completed

market_data


Loading ITables v2.4.3 from the internet... (need help?)


## Macro data


In [10]:
from fredapi import Fred
import pandas as pd

fred = Fred(api_key='c2c6e01a4f067abfc92b53024fb3e4c2')
start_date = '2000-01-01'

# Download each macroeconomic indicator as a DataFrame
indicators = {
    'CPI': 'CPIAUCSL',
    'Fed_Funds_Rate': 'FEDFUNDS',
    'M2_Supply': 'M2SL',
    'Unemployment': 'UNRATE',
    'Treasury_Yield_10Y': 'GS10',
    'Real_Interest_Rate': 'DFII10',
    'USD_Index': 'DTWEXBGS'
}

macro_monthly = pd.DataFrame()
real_interest_rate = pd.DataFrame()
usd_index = pd.DataFrame()

for name, code in indicators.items():
    series = fred.get_series(code, observation_start=start_date)
    series = series.to_frame(name=name)
    series.index.name = 'Date'
    if name == 'USD_Index':
        usd_index = series
    elif name == 'Real_Interest_Rate':
        real_interest_rate = series
    else:
        if macro_monthly.empty:
            macro_monthly = series
        else:
            macro_monthly = macro_monthly.join(series, how='outer')

macro_monthly.to_csv("../data/raw/macro_monthly.csv")
real_interest_rate.to_csv("../data/raw/real_interest_rate.csv")
usd_index.to_csv("../data/raw/usd_index.csv")




## VIX data


In [ ]:
import yfinance as yf 

vix = yf.download("^VIX", start="2000-01-01")
close_vix = vix["Close"]

close_vix.to_csv("data/vix_data.csv")

print(vix.head())

[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open Volume
Ticker           ^VIX       ^VIX       ^VIX       ^VIX   ^VIX
Date                                                         
2000-01-03  24.209999  26.150000  23.980000  24.360001      0
2000-01-04  27.010000  27.180000  24.799999  24.940001      0
2000-01-05  26.410000  29.000000  25.850000  27.980000      0
2000-01-06  25.730000  26.709999  24.700001  26.680000      0
2000-01-07  21.719999  25.170000  21.719999  25.139999      0
